In [1]:
#Loading in our data
!wget https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Gift_Cards_5.json.gz --no-check-certificate
!wget https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Gift_Cards.csv --no-check-certificate
!gzip -d Gift_Cards_5.json.gz
!head Gift_Cards_5.json
!head Gift_Cards.csv
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Gift_Cards.json.gz --no-check-certificate
!gzip -d meta_Gift_Cards.json.gz
!head meta_Gift_Cards.json

--2023-11-07 10:41:30--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Gift_Cards_5.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176952 (173K) [application/x-gzip]
Saving to: ‘Gift_Cards_5.json.gz’

Gift_Cards_5.json.g 100%[===================>] 172.80K  --.-KB/s    in 0.02s   

2023-11-07 10:41:30 (7.94 MB/s) - ‘Gift_Cards_5.json.gz’ saved [176952/176952]

--2023-11-07 10:41:30--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Gift_Cards.csv
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6143525 (5.9M) [text/csv]
Saving to: ‘Gift_Cards.csv.1’

Gift_Cards.csv.1    100%[===================>]   5.86M  15.0MB/s    in 0.4s    

2023-11-07 

In [1]:
#Installing packages that do not come pre installed
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.1+cu116.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.1+cu116.html
!pip install torch-geometric
!pip install umap-learn

Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp39-cp39-macosx_11_0_arm64.whl size=268030 sha256=e6f251fa9f1c62a6980349854399a7c2fbc75a25352965f8ba1b45597358068e
  Stored in directory: /Users/aidenchang/Library/Caches/pip/wheels/b8/9c/c0/d261e1e64cc49e6b14dd7319d8b61eb14168529d810dbba00e
Successfully built torch-scatter
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp39-cp39-macosx_11_0_arm64.whl size=463278 sha256=648402f2557253b592a7e08fbc561f3cd1bbb375b68363cef3ab2f02085bcc8c
  Stored in directory: /Users/aidenchang/Library/Caches/pip/wheels/c0/15/18/881c60adefeb60882987c23aa5ecced0ab7257c3162acab518
Successfully built torch-sparse
  Us

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 41.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.3 MB/s eta 0:00:00a 0:00:01


In [1]:
# General libraries
import json
from pathlib import Path as Data_Path
import os
from os.path import isfile, join
import pickle
import random

import numpy as np
import networkx as nx
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm

In [2]:
from typing import Optional, Union

import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import Embedding, ModuleList, Linear
import torch.nn.functional as F

import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from torch_geometric.transforms import RandomLinkSplit
from torch.nn.modules.loss import _Loss

from torch_geometric.nn.conv import LGConv, GATConv, SAGEConv
from torch_geometric.typing import Adj, OptTensor, SparseTensor


print(f"Torch version: {torch.__version__}; Torch-cuda version: {torch.version.cuda}; Torch Geometric version: {torch_geometric.__version__}.")

/opt/anaconda3/envs/pytorchG/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 2.1.0; Torch-cuda version: None; Torch Geometric version: 2.4.0.


In [3]:
# set the seed for reproducibility
seed = 224
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [4]:
##Defining classes

class Product:
  def __init__(self, title, asin, description, also_bought, categories):
    self.title = title #name, string
    self.asin = asin #string
    self.description = description #string
    # self.features = features # String[]
    self.also_bought = also_bought #asin[]
    self.categories = set(categories) #categories[]


  def __str__(self):
    return f"Product {self.title}"

class Category:
  def __init__(self, name):
    self.name = name
    self.products = []

  def add_to_category(self, product):
    self.products.append(product)

  def __str__(self):
    return f"Category {self.name} has {len(self.products)} in it."

class User:
  def __init__(self, reviewerID):
    self.reviewerID = reviewerID
    self.products_reviewed = []
    #maybe we can add the categories that they reviewed?

  def add_to_reviewed(self, product):
    self.products_reviewed.append(product)

  def __str__(self):
    return f"User {self.reviewerID} has reviewed {len(self.products_reviewed)} products"

In [5]:
import pandas as pd
import json

# reviews = pd.read_csv("Gift_Cards.csv")
review_data = []
review_name = 'Gift_Cards_5.json'
with open(review_name, 'r') as file:
    for line in file:
      review_data.append(json.loads(line))
file_metadata = []
metadata_link = "meta_Gift_Cards.json"
with open(metadata_link, 'r') as file:
    for line in file:
      file_metadata.append(json.loads(line))

In [6]:
review_data[1]

{'overall': 4.0,
 'verified': True,
 'reviewTime': '06 14, 2018',
 'reviewerID': 'A2MN5JQMIY0FQ2',
 'asin': 'B004LLIKVU',
 'style': {'Gift Amount:': ' 50'},
 'reviewerName': 'Amazon Customer',
 'reviewText': 'Gift card for my daughter',
 'summary': 'Ease of use',
 'unixReviewTime': 1528934400}

In [7]:
products = {}
all_categories = {}
users = {}

def process_review_data(review):
  try:
    userID = review['reviewerID']
  except:
    print(review)
  if userID not in users:
    new_user = User(userID)
    users[userID] = new_user
  rating = int(review['overall'])
  users[userID].add_to_reviewed((review['asin'], rating))

def process_metadata(metadata):
  asin = metadata['asin']
  # features = metadata['features']
  also_bought = metadata['also_buy']
  description = metadata['description']
  categories = metadata['category']
  title = metadata['title']
  if asin not in products:
    new_product = Product(title, asin, description, also_bought, categories)
    products[asin] = new_product
  for category in categories:
    if category not in all_categories:
      all_categories[category] = Category(category)
    all_categories[category].add_to_category(asin)


def load_data():
  for line in review_data:
    process_review_data(line)
  for line in file_metadata:
    process_metadata(line)


In [8]:
load_data()

In [9]:
import torch
from torch_geometric.data import Data

# Create instances of your classes
# This is just a placeholder. Replace this with your actual data.
# products = [...]
# categories = [...]
# users = [...]

# Create mappings from unique IDs to consecutive integers
unique_user_id = [user for user in users]
unique_user_id = pd.DataFrame(data={
   'userId': unique_user_id,
   'mappedID': pd.RangeIndex(len(unique_user_id)),
})

unique_product_id = [product for product in products]
unique_product_id = pd.DataFrame(data={
   'productId': unique_product_id,
   'mappedID': pd.RangeIndex(len(unique_product_id)),
})

unique_category_id = [category for category in all_categories]
unique_category_id = pd.DataFrame(data={
   'categoryId': unique_category_id,
   'mappedID': pd.RangeIndex(len(unique_category_id)),
})
# print(unique_user_id)
# Construct the edge indices
# User to Product (Review)
user_product_edges = []
user_product_ratings = []
for user_name in users:
   user_object = users[user_name]
   for product_id, rating in users[user_name].products_reviewed:
      product = products[product_id]
      user_product_edges.append([unique_user_id.loc[unique_user_id['userId'] == user_object.reviewerID, 'mappedID'].values[0],
                              unique_product_id.loc[unique_product_id['productId'] == product.asin, 'mappedID'].values[0]])
      user_product_ratings.append(rating)
user_product_edges = torch.t(torch.tensor(user_product_edges, dtype=torch.long))
user_product_ratings = torch.tensor(user_product_ratings, dtype=torch.float) 
# user_product_ratings = torch.reshape(user_product_ratings, (1, len(user_product_ratings)))

print(user_product_edges.shape)

# Product to Product (Also Bought)
product_product_edges = []
for product_id in products:
   product = products[product_id]
   for also_bought_product in products[product_id].also_bought:
      #  print(also_bought_product)
 
       if (unique_product_id['productId'] == also_bought_product).any():  
         product_product_edges.append([unique_product_id.loc[unique_product_id['productId'] == product.asin, 'mappedID'].values[0],
                                   unique_product_id.loc[unique_product_id['productId'] == also_bought_product, 'mappedID'].values[0]])
# print(len(product_product_edges))
product_product_edges = torch.t(torch.tensor(product_product_edges, dtype=torch.long))

# Product to Category
product_category_edges = []
for product_id in products:
   product = products[product_id]
   for category in products[product_id].categories:
       product_category_edges.append([unique_product_id.loc[unique_product_id['productId'] == product.asin, 'mappedID'].values[0],
                                    unique_category_id.loc[unique_category_id['categoryId'] == category, 'mappedID'].values[0]])

product_category_edges = torch.t(torch.tensor(product_category_edges, dtype=torch.long))

torch.Size([2, 2972])


In [10]:
print(len(product_product_edges), len(product_category_edges), len(product_category_edges))



2 2 2


In [11]:
from sentence_transformers import SentenceTransformer

class TextEncoder:
    """
    A class for encoding text using a SentenceTransformer model.
    """
    def __init__(self, model='all-MiniLM-L6-v2', device=None):
        """
        :param model: Name of the SentenceTransformer model to use.
        :param device: Device to use for model inference. Default is None.
        """
        self.device = device
        self.model = SentenceTransformer(model, device=self.device)

    @torch.no_grad()
    def __call__(self, values: list):
        """
        Encode a list of text values into embeddings.

        :param values: List of text values to encode.
        :return: Encoded embeddings as a PyTorch tensor.
        """
        x = self.model.encode(values,
                              show_progress_bar=True,
                              convert_to_tensor=True,
                              device=self.device)
        return x.cpu()


# Check if CUDA is available, and set the device accordingly
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# Create an instance of the TextEncoder class with the determined device
encoder = TextEncoder(device=device)

In [12]:
ex = "I am aiden"
e = encoder(ex)
print(e.shape)


Batches: 100%|██████████| 1/1 [00:00<00:00, 33.42it/s]

torch.Size([384])


In [13]:
products[list(products.keys())[0]].description

['Gift card for the purchase of goods or services at Serendipity 3 in New York City only. Not valid for online purchases.',
 'Statements regarding dietary supplements have not been evaluated by the FDA and are not intended to diagnose, treat, cure, or prevent any disease or health condition.']

In [14]:
# Create product features
product_features = torch.zeros((len(unique_product_id), 384))
for product_id in products:
   product = products[product_id]
   mapped_id = unique_product_id.loc[unique_product_id['productId'] == product.asin, 'mappedID'].values[0]
   d = ""
   for descrip in product.description:
      d = d + descrip
   product.description = d
   product_features[mapped_id] = encoder(product.description)

Batches: 100%|██████████| 1/1 [00:00<00:00, 17.07it/s]


In [15]:
product_features[-1].size()

torch.Size([384])

In [16]:
from torch_geometric.data import HeteroData
data = HeteroData()

# Add node features to the HeteroData object
data['Product'].x = product_features
data['User'].x = torch.ones(len(users), 1)
data['Category'].x = torch.ones(len(all_categories), 1)

# Add edge indices to the HeteroData object
data['User', 'REVIEWS', 'Product'].edge_index = user_product_edges
data['User', 'REVIEWS', 'Product'].edge_attr = user_product_ratings
data['Product', 'ALSO_BOUGHT', 'Product'].edge_index = product_product_edges
data['Product', 'HAS_CATEGORY', 'Category'].edge_index = product_category_edges


import torch_geometric.transforms as T
data = T.ToUndirected()(data)

In [19]:
# data['Product'].x = product_features
# data['User'].x = user_features

In [69]:

# n, e = data.metadata()
# print(n)
# print(e)
# data['Category'].num_nodes



In [121]:
data

HeteroData(
  Product={ x=[1547, 384] },
  User={ x=[458, 1] },
  Category={ x=[14, 1] },
  (User, REVIEWS, Product)={
    edge_index=[2, 2972],
    edge_attr=[2972],
  },
  (Product, ALSO_BOUGHT, Product)={ edge_index=[2, 10572] },
  (Product, HAS_CATEGORY, Category)={ edge_index=[2, 3622] },
  (Product, rev_REVIEWS, User)={
    edge_index=[2, 2972],
    edge_attr=[2972],
  },
  (Category, rev_HAS_CATEGORY, Product)={ edge_index=[2, 3622] }
)

In [17]:
import torch_geometric.transforms as T


# t_data = T.ToUndirected()(data)
# t_data = T.ToUndirected()(data)
# t_data = T.ToUndirected()(data)

transform = RandomLinkSplit(
    is_undirected=False, 
    add_negative_train_samples=False, 
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=0,
    num_val=0.1, num_test=0.1,
    edge_types=["REVIEWS","rev_REVIEWS"]
)

#we have to do the split for each edge type and combine them back together

# transform = RandomLinkSplit(is_undirected=True, edge_types=["REVIEWS","ALSO_BOUGHT", 'HAS_CATEGORY'])
train_split, val_split, test_split = transform(data)
# train_split, val_split, test_split = transform(data)

In [72]:
torch.Tensor.type(train_split['User', 'REVIEWS', 'Product'].edge_index[0]) #64bit int
# type(test_split['User', 'REVIEWS', 'Product'].edge_index[0][0])

'torch.LongTensor'

In [42]:
from torch_geometric.utils import negative_sampling
n = negative_sampling(train_split['User', 'REVIEWS', 'Product'].edge_index, num_neg_samples=1)
print(train_split['User', 'REVIEWS', 'Product'].edge_index, n)

tensor([[ 435,  145,   93,  ...,  137,  312,  388],
        [ 921, 1231,  194,  ...,  308, 1232, 1231]]) tensor([[  32],
        [1199]])


In [73]:
data

HeteroData(
  Product={ x=[1547, 384] },
  User={ x=[458, 1] },
  Category={ x=[14, 1] },
  (User, REVIEWS, Product)={
    edge_index=[2, 2972],
    edge_attr=[2972],
  },
  (Product, ALSO_BOUGHT, Product)={ edge_index=[2, 10572] },
  (Product, HAS_CATEGORY, Category)={ edge_index=[2, 3622] },
  (Product, rev_REVIEWS, User)={
    edge_index=[2, 2972],
    edge_attr=[2972],
  },
  (Category, rev_HAS_CATEGORY, Product)={ edge_index=[2, 3622] }
)

In [18]:
num_users = len(data['User']['x'])
num_products = data['Product']['x'].shape[0]
# print(num_users, num_products)
# assert(num_products == 1547)
positive_users, positive_products = data['REVIEWS']['edge_index']
print(positive_users)
mask = torch.zeros(num_users, num_products, device=device, dtype=torch.bool)
mask[positive_users, positive_products] = True
flat_mask = mask.flatten()
negative_indices = torch.where(~flat_mask)[0]
print(negative_indices)





tensor([  0,   0,   0,  ..., 457, 457, 457])
tensor([     0,      1,      2,  ..., 708522, 708523, 708524])


In [19]:
def sample_negative_edges_hetero(data, edge_type, num_nodes_1, num_nodes_2, device=None):
    # Extract the positive edges for the given edge type
    positive_edges = data[edge_type]['edge_index']

    # Create a mask tensor with the appropriate shape
    mask = torch.zeros(num_nodes_1, num_nodes_2, device=device, dtype=torch.bool)
    mask[positive_edges[0], positive_edges[1]] = True

    # Flatten the mask tensor and get the indices of the negative edges
    flat_mask = mask.flatten()
    negative_indices = torch.where(~flat_mask)[0]

    # Sample negative edges from the negative_indices tensor
    sampled_negative_indices = negative_indices[
        torch.randint(0, negative_indices.size(0), size=(positive_edges.size(1),), device=device)
    ]

    # Convert the indices back to the two types of nodes
    nodes_1 = torch.div(sampled_negative_indices, num_nodes_2, rounding_mode='floor')
    nodes_2 = sampled_negative_indices % num_nodes_2

    neg_edge_index = torch.stack((nodes_1, nodes_2), dim=0)
    neg_edge_label = torch.zeros(neg_edge_index.shape[1], device=device)

    return neg_edge_index, neg_edge_label

In [20]:
class BPRLoss(_Loss):
    r"""The Bayesian Personalized Ranking (BPR) loss.

    The BPR loss is a pairwise loss that encourages the prediction of an
    observed entry to be higher than its unobserved counterparts
    (see `here <https://arxiv.org/abs/2002.02126>`__).

    .. math::
        L_{\text{BPR}} = - \sum_{u=1}^{M} \sum_{i \in \mathcal{N}_u}
        \sum_{j \not\in \mathcal{N}_u} \ln \sigma(\hat{y}_{ui} - \hat{y}_{uj})
        + \lambda \vert\vert \textbf{x}^{(0)} \vert\vert^2

    where :math:`lambda` controls the :math:`L_2` regularization strength.
    We compute the mean BPR loss for simplicity.

    Args:
        lambda_reg (float, optional): The :math:`L_2` regularization strength
            (default: 0).
        **kwargs (optional): Additional arguments of the underlying
            :class:`torch.nn.modules.loss._Loss` class.
    """
    __constants__ = ['lambda_reg']
    lambda_reg: float

    def __init__(self, lambda_reg: float = 0, **kwargs):
        super().__init__(None, None, "sum", **kwargs)
        self.lambda_reg = lambda_reg

    def forward(self, positives: Tensor, negatives: Tensor,
                parameters: Tensor = None) -> Tensor:
        r"""Compute the mean Bayesian Personalized Ranking (BPR) loss.

        .. note::

            The i-th entry in the :obj:`positives` vector and i-th entry
            in the :obj:`negatives` entry should correspond to the same
            entity (*.e.g*, user), as the BPR is a personalized ranking loss.

        Args:
            positives (Tensor): The vector of positive-pair rankings.
            negatives (Tensor): The vector of negative-pair rankings.
            parameters (Tensor, optional): The tensor of parameters which
                should be used for :math:`L_2` regularization
                (default: :obj:`None`).
        """
        n_pairs = positives.size(0)
        log_prob = F.logsigmoid(positives - negatives).sum()
        regularization = 0

        if self.lambda_reg != 0:
            # parameters = torch.tensor(list(parameters))
            # regularization = self.lambda_reg * parameters.norm(p=2).pow(2) 
            regularization = sum(self.lambda_reg * p.norm(p=2).pow(2) for p in parameters)

        return (-log_prob + regularization) / n_pairs

In [96]:
from torch_geometric.nn import LightGCN

model = LightGCN(
    num_nodes=train_split.num_nodes,
    embedding_dim=64,
    num_layers=2,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
bpr_loss = BPRLoss(lambda_reg=0.01).to(device)


In [21]:
def recall_at_k_hetero(data, model, edge_types=None, k=10, batch_size=64, device=None):
    if not edge_types:
        edge_types = [
            ('User', 'REVIEWS', 'Product'),
            ('Product', 'rev_REVIEWS', 'User')
        ]

    recalls = []

    # Compute embeddings for all node types
    with torch.no_grad():
        embeddings = {node_type: model.get_embedding(data[node_type].x.to(device))
                      for node_type in data.node_types}

    for edge_type in edge_types:
        source_node_type, _, target_node_type = edge_type

        n_sources = data[source_node_type].x.size(0)
        n_targets = data[target_node_type].x.size(0)

        source_embeddings = embeddings[source_node_type]
        target_embeddings = embeddings[target_node_type]

        hits_list = []
        relevant_counts_list = []

        for batch_start in range(0, n_sources, batch_size):
            batch_end = min(batch_start + batch_size, n_sources)
            batch_source_embeddings = source_embeddings[batch_start:batch_end]

            # Calculate scores for all possible source-target pairs
            scores = torch.matmul(batch_source_embeddings, target_embeddings.t())

            # Find the top k highest scoring targets for each source in the batch
            _, top_k_indices = torch.topk(scores, k, dim=1)

            # Ground truth supervision edges
            ground_truth_edges = data[edge_type].edge_label_index.to(device)

            # Create a mask to indicate if the top k targets are in the ground truth supervision edges
            mask = torch.zeros(scores.shape, device=device, dtype=torch.bool)
            mask[ground_truth_edges[0], ground_truth_edges[1]] = True

            # Check how many of the top k targets are in the ground truth supervision edges
            hits = mask.gather(1, top_k_indices).sum(dim=1)
            hits_list.append(hits)

            # Calculate the total number of relevant targets for each source in the batch
            relevant_counts = torch.bincount(ground_truth_edges[0], minlength=batch_end - batch_start)
            relevant_counts_list.append(relevant_counts)

        # Compute recall@k for this edge type
        hits_tensor = torch.cat(hits_list, dim=0)
        relevant_counts_tensor = torch.cat(relevant_counts_list, dim=0)
        recall_at_k = torch.true_divide(hits_tensor, relevant_counts_tensor.clamp(min=1))
        recall_at_k = recall_at_k.mean().item()

        recalls.append(recall_at_k)

    # Return the average recall across all edge types
    overall_recall = sum(recalls) / len(recalls)
    return overall_recall



In [ ]:
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
num_epochs = 500
edge_types = [
    ('User', 'REVIEWS', 'Product'),
    ('Product', 'rev_REVIEWS', 'User')
]

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    total_loss = 0
    for edge_type in edge_types:
        node_type_1, relation, node_type_2 = edge_type
        num_nodes_1 = train_split[node_type_1]['x'].size(0)
        num_nodes_2 = train_split[node_type_2]['x'].size(0)

        # Sample negative edges
        neg_edge_index, _ = sample_negative_edges_hetero(
            train_split, relation, num_nodes_1, num_nodes_2, device=device
        )

        # Get the positive edges
        pos_edge_index = train_split[edge_type]['edge_index'].to(device)

        # Compute scores for positive and negative edges
        # print(pos_edge_index)
        pos_scores = model(pos_edge_index)
        neg_scores = model(neg_edge_index)

        # Calculate BPR loss for this edge type
        loss = bpr_loss(pos_scores, neg_scores, model.parameters())
        total_loss += loss

    # Backward pass
    total_loss.backward()
    optimizer.step()

    # Print loss every epoch (or every few epochs)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss.item()}")


# GAT

In [22]:
data

HeteroData(
  Product={ x=[1547, 384] },
  User={ x=[458, 1] },
  Category={ x=[14, 1] },
  (User, REVIEWS, Product)={
    edge_index=[2, 2972],
    edge_attr=[2972],
  },
  (Product, ALSO_BOUGHT, Product)={ edge_index=[2, 10572] },
  (Product, HAS_CATEGORY, Category)={ edge_index=[2, 3622] },
  (Product, rev_REVIEWS, User)={
    edge_index=[2, 2972],
    edge_attr=[2972],
  },
  (Category, rev_HAS_CATEGORY, Product)={ edge_index=[2, 3622] }
)

In [58]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import HGTConv

In [73]:
# node_feature_sizes = {
#     'Product': 384,
#     'User': 1,
#     'Category': 1
# }

# class HGTModel(torch.nn.Module):
#     def __init__(self, hidden_channels, out_channels, num_heads, num_layers, node_types, edge_types, metadata):
#         super(HGTModel, self).__init__()
#         self.metadata = metadata
#         self.lin_dict = torch.nn.ModuleDict()
#         for node_type, in_features in node_feature_sizes.items():
#             self.lin_dict[node_type] = Linear(in_features, hidden_channels)

#         # Initialize HGT convolutional layers
#         self.convs = torch.nn.ModuleList()
#         for _ in range(num_layers):
#             conv = HGTConv(hidden_channels, hidden_channels, edge_types,
#                            num_heads=num_heads, group='sum')
#             self.convs.append(conv)

#         # Final linear transformation
#         self.final_lin = Linear(hidden_channels, out_channels)

#     def forward(self, x_dict, edge_index_dict):
#         # Apply initial linear transformations and ReLU activation
#         x_dict = {node_type: self.lin_dict[node_type](x).relu_()
#                   for node_type, x in x_dict.items()}

#         # Apply HGT convolutional layers
#         for conv in self.convs:
#             x_dict = conv(x_dict, edge_index_dict)

#         # Compute embeddings for User and Product nodes
#         user_embedding = x_dict['User']
#         product_embedding = x_dict['Product']

#         return user_embedding, product_embedding

In [68]:
# hidden_channels = 64
# out_channels = 64  # This can be the same as hidden_channels for embedding purposes
# num_heads = 2
# num_layers = 1

# # Node types and edge types in your data
# node_types = ['Product', 'User', 'Category']
# edge_types = [('User', 'REVIEWS', 'Product'), ('Product', 'ALSO_BOUGHT', 'Product'), 
#               ('Product', 'HAS_CATEGORY', 'Category'), ('Product', 'rev_REVIEWS', 'User'), 
#               ('Category', 'rev_HAS_CATEGORY', 'Product')]

# # Model instantiation
# model = HGTModel(hidden_channels, out_channels, num_heads, num_layers, node_types, edge_types, metadata = data.metadata())


In [86]:
model = HGTConv(
    in_channels=-1,
    out_channels=64,
    edge_types=[('User', 'REVIEWS', 'Product'), ('Product', 'ALSO_BOUGHT', 'Product'), 
              ('Product', 'HAS_CATEGORY', 'Category'), ('Product', 'rev_REVIEWS', 'User'), 
              ('Category', 'rev_HAS_CATEGORY', 'Product')],
    num_heads=5,
    metadata=data.metadata()
).to(device)


In [92]:
def train(model, data, optimizer, device, edge_types):
    model.train()

    optimizer.zero_grad()
    total_loss = 0

    for edge_type in edge_types:
        node_type_1, relation, node_type_2 = edge_type
        num_nodes_1 = data[node_type_1]['x'].size(0)
        num_nodes_2 = data[node_type_2]['x'].size(0)

        # Sample negative edges for this edge type
        neg_edge_index, _ = sample_negative_edges_hetero(
            data, edge_type, num_nodes_1, num_nodes_2, device=device
        )
        # print('neg_edge_index', neg_edge_index)
        # Get the embeddings for all nodes
        
        x_dict, edge_index_dict = data.x_dict, data.edge_index_dict
        # for key, value in x_dict.items():
        #     print(f"x_dict[{key}].shape: {value.shape}, x_dict[{key}]: {value}")

        # for key, value in edge_index_dict.items():
        #     print(f"edge_index_dict[{key}].shape: {value.shape}, edge_index_dict[{key}]: {value}")
        
        embeddings = model(x_dict, edge_index_dict)

        # Extract embeddings for positive edges
        pos_edge_index = data[edge_type]['edge_index'].to(device)
        pos_scores = (embeddings[node_type_1][pos_edge_index[0]] * embeddings[node_type_2][pos_edge_index[1]]).sum(dim=-1)

        # Extract embeddings for negative edges
        neg_scores = (embeddings[node_type_1][neg_edge_index[0]] * embeddings[node_type_2][neg_edge_index[1]]).sum(dim=-1)

        # Compute BPR loss for this edge type
        bpr_loss_func = BPRLoss(lambda_reg=0.001)
        loss = bpr_loss_func(pos_scores, neg_scores, model.parameters())
        total_loss += loss

    # Backward pass and optimize
    total_loss.backward()
    optimizer.step()

    return total_loss.item()


In [94]:
num_epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [95]:
for epoch in range(num_epochs):
    loss = train(model, data, optimizer, device, edge_types)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}')

Epoch 1/10, Loss: 4.0044
Epoch 2/10, Loss: 3.0130
Epoch 3/10, Loss: 1.8745
Epoch 4/10, Loss: 1.7030
Epoch 5/10, Loss: 1.4415
Epoch 6/10, Loss: 1.1031
Epoch 7/10, Loss: 1.0380
Epoch 8/10, Loss: 0.9535
Epoch 9/10, Loss: 1.0154
Epoch 10/10, Loss: 0.8669


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [104]:
def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        embeddings = model(data.x_dict, data.edge_index_dict)

        edge_index = data['User', 'REVIEWS', 'Product'].edge_label_index
        scores = (embeddings['User'][edge_index[0]] * embeddings['Product'][edge_index[1]]).sum(dim=-1)
        
        labels = data['User', 'REVIEWS', 'Product'].edge_label
        auc_roc = roc_auc_score(labels.cpu().numpy(), scores.cpu().numpy())

    return auc_roc


In [108]:
from copy import deepcopy

val_split_copy = deepcopy(val_split)

In [110]:
val_split_copy

HeteroData(
  Product={ x=[1547, 384] },
  User={ x=[458, 1] },
  Category={ x=[14, 1] },
  (User, REVIEWS, Product)={
    edge_index=[2, 2378],
    edge_attr=[2378],
    edge_label=[297],
    edge_label_index=[2, 297],
  },
  (Product, ALSO_BOUGHT, Product)={ edge_index=[2, 10572] },
  (Product, HAS_CATEGORY, Category)={ edge_index=[2, 3622] },
  (Product, rev_REVIEWS, User)={
    edge_index=[2, 2378],
    edge_attr=[2378],
    edge_label=[297],
    edge_label_index=[2, 297],
  },
  (Category, rev_HAS_CATEGORY, Product)={ edge_index=[2, 3622] }
)

In [116]:
num_nodes_1 = val_split_copy['User']['x'].size(0)
num_nodes_2 = val_split_copy['Product']['x'].size(0)

In [117]:
neg_edge_index, neg_edge_label = sample_negative_edges_hetero(
   val_split_copy, 
   ('User', 'REVIEWS', 'Product'), 
   num_nodes_1, 
   num_nodes_2, 
   device=device
)

In [131]:
edge_index = val_split_copy['User', 'REVIEWS', 'Product'].edge_label_index
edge_labels = val_split_copy['User', 'REVIEWS', 'Product'].edge_label

In [132]:
all_edge_index = torch.cat([edge_index, neg_edge_index], dim=1)
all_edge_label = torch.cat([edge_labels, neg_edge_label], dim=0)

In [134]:
def evaluate_with_negatives(model, data, all_edge_index, all_edge_label):
    model.eval()
    with torch.no_grad():
        embeddings = model(data.x_dict, data.edge_index_dict)

        # Use 'all_edge_index' for scores instead of 'edge_index'
        scores = (embeddings['User'][all_edge_index[0]] * embeddings['Product'][all_edge_index[1]]).sum(dim=-1)

        # Use 'all_edge_label' instead of 'labels'
        auc_roc = roc_auc_score(all_edge_label.cpu().numpy(), scores.cpu().numpy())

    return auc_roc

In [135]:
test = evaluate_with_negatives(model, val_split_copy, all_edge_index, all_edge_label)

In [136]:
test

0.9516853423497662